In [1]:
!unzip /content/drive/MyDrive/mil_ird_challenge.zip

Archive:  /content/drive/MyDrive/mil_ird_challenge.zip
replace mil_ird_challenge/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
import os
import pandas as pd

train_root_path = '/content/mil_ird_challenge/train/'
test_root_path = '/content/mil_ird_challenge/test/'

train_set = []
test_set = []
for cls in os.listdir(train_root_path):
  cls_path = os.path.join(train_root_path,cls)
  if os.path.isdir(cls_path):
    for fname in os.listdir(cls_path):
      f_low = fname.lower()
      if f_low.endswith('.png') or f_low.endswith('.jpg'):
        f_path = os.path.join(cls_path, fname)
        train_set.append((cls, f_path))

for cls in os.listdir(test_root_path):
  cls_path = os.path.join(test_root_path,cls)
  if os.path.isdir(cls_path):
    for fname in os.listdir(cls_path):
      f_low = fname.lower()
      if f_low.endswith('.png') or f_low.endswith('.jpg'):
        f_path = os.path.join(cls_path, fname)
        test_set.append((cls, f_path))

train_df = pd.DataFrame(data=train_set, columns=['class', 'path'])
test_df = pd.DataFrame(data=test_set, columns=['class', 'path'])
print(train_df.shape)
print(test_df.shape)

(13953, 2)
(3000, 2)


In [6]:
train_df.groupby('class').count().describe()

,path
count,6.000000
mean,2325.500000
std,117.716184
min,2175.000000
25%,2257.000000
50%,2309.500000
75%,2393.500000
max,2498.000000


In [4]:
#!git clone https://github.com/facebookresearch/FixRes.git

In [2]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import time, copy
#from FixRes.transforms_v2 import get_transforms

def set_parameter_requires_grad(model, feature_extracting):
  if feature_extracting:
    for param in model.parameters():
      param.requires_grad = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

#transform = get_transforms(input_size=480, test_size=480)
#act_train_set = datasets.ImageFolder(train_root_path, transform=transform['val_train'])
#act_test_set = datasets.ImageFolder(test_root_path, transform=transform['val_test'])

batch_size = 32
n_epochs = 30
n_classes = train_df['class'].nunique()

model_ft = models.resnet50(pretrained=True)
set_parameter_requires_grad(model_ft, feature_extracting=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, n_classes)
input_size = 224


In [3]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
act_train_set = datasets.ImageFolder(train_root_path, transform=data_transforms['train'])
act_test_set = datasets.ImageFolder(test_root_path, transform=data_transforms['val'])
image_datasets = {'train': act_train_set, 'val': act_test_set}

dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Initial Optimizer and finetuning

In [5]:
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
params_to_update = []
for name,param in model_ft.named_parameters():
  if param.requires_grad == True:
    params_to_update.append(param)
    print("\t",name)

optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.99)
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=n_epochs, is_inception=False)

	 fc.weight
	 fc.bias
Epoch 0/29
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.6941 Acc: 0.7881
val Loss: 0.3389 Acc: 0.8970

Epoch 1/29
----------
train Loss: 0.5971 Acc: 0.8203
val Loss: 0.3954 Acc: 0.8767

Epoch 2/29
----------
train Loss: 0.5771 Acc: 0.8266
val Loss: 0.3427 Acc: 0.8923

Epoch 3/29
----------
train Loss: 0.5870 Acc: 0.8230
val Loss: 0.5427 Acc: 0.8507

Epoch 4/29
----------
train Loss: 0.5947 Acc: 0.8276
val Loss: 0.3146 Acc: 0.9067

Epoch 5/29
----------
train Loss: 0.4908 Acc: 0.8377
val Loss: 0.3949 Acc: 0.8837

Epoch 6/29
----------
train Loss: 0.4909 Acc: 0.8398
val Loss: 0.3029 Acc: 0.9007

Epoch 7/29
----------
train Loss: 0.5564 Acc: 0.8312
val Loss: 0.3368 Acc: 0.8963

Epoch 8/29
----------
train Loss: 0.4927 Acc: 0.8383
val Loss: 0.3859 Acc: 0.8867

Epoch 9/29
----------
train Loss: 0.4942 Acc: 0.8408
val Loss: 0.2611 Acc: 0.9150

Epoch 10/29
----------
train Loss: 0.5185 Acc: 0.8433
val Loss: 0.2891 Acc: 0.9073

Epoch 11/29
----------
train Loss: 0.4761 Acc: 0.8453
val Loss: 0.3438 Acc: 0.8813

Epoch 12/29
----------
t

In [9]:
torch.save(model_ft.state_dict(), '/content/drive/MyDrive/trained/model.pt')
torch.save(model_ft.state_dict(), 'model.pt')

In [19]:
from PIL import Image

preprocess = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

pred_root_path = '/content/mil_ird_challenge/pred/'
#pred_set = datasets.ImageFolder(pred_root_path, transform=data_transforms['val'])
pred_labels = []
model_ft.eval()
for fname in os.listdir(pred_root_path):
  fpath = os.path.join(pred_root_path, fname)
  img = Image.open(fpath)
  img_p = preprocess(img)
  img_p_tensor = torch.unsqueeze(img_p, 0).cuda()
  #display(img_p)
  pred_lbl = torch.argmax(model_ft(img_p_tensor)).item()
  pred_labels.append([fname, pred_lbl])
  #print(pred_lbl)
  #break
#print(pred_labels)
res_df = pd.DataFrame(data = pred_labels, columns=['filename', 'class_prediction'])
display(res_df)
res_df.to_csv('prediction_result.csv')
res_df.to_csv('/content/drive/MyDrive/trained/prediction_result.csv')

,filename,class_prediction
0,17410.jpg,4
1,1763.jpg,0
2,17315.jpg,5
3,17600.jpg,1
4,1730.jpg,5
...,...,...
76,174.jpg,4
77,17347.jpg,5
78,17590.jpg,0
79,17673.jpg,0
